# Exercise 3

In [1]:
import numpy as np
import pandas as pd

## PART A
**(15 points)** If we use the weighted entropy:
$${\displaystyle \operatorname {Weighted\ Entropy(F,D)} = \sum_{f \in values(F)}\frac{|D_f|}{|D|}H(D_f)}$$

instead of the information gain:
$${\displaystyle \operatorname {Gain(F,D)} =H(D) -\sum_{f \in values(F)}\frac{|D_f|}{|D|}H(D_f)}$$

to determine the best feature, will that work? And if so, does that change how the next best feature for splitting is selected?

## PART B
* **(60 points)** Study the detailed example of the `07.decision_trees.ipynb` handout, and follow its steps to train a decision tree on the dataset below. The goal is to be able to recommend whether to play golf or not based on weather conditions (outlook, temperature, and humidity). Use the **weighted gini index** instead of the information gain to construct the tree. 

In [2]:
df = pd.DataFrame({
    'Outlook': ['Rainy', 'Overcast', 'Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Sunny', 'Rainy', 'Overcast', 'Overcast'],
    'Temperature': ['Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot'],
    'Humidity': ['High', 'High', 'High', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal'],
    'Play Golf': ['No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']
})

df

,Outlook,Temperature,Humidity,Play Golf
0,Rainy,Hot,High,No
1,Overcast,Hot,High,Yes
2,Sunny,Mild,High,Yes
3,Sunny,Cool,Normal,No
4,Overcast,Cool,Normal,Yes
5,Rainy,Mild,High,No
6,Rainy,Cool,Normal,Yes
7,Sunny,Mild,Normal,Yes
8,Rainy,Mild,Normal,Yes
9,Overcast,Mild,High,Yes


  Theoretically, the Gini impurity for a set of examples with labels $y_i$, where $i \in \{1, 2,\dots,L\}$ and $L$ is the number of unique labels, is written as:

  $${\displaystyle \operatorname {G(D)}=\sum _{i=1}^{L}\sum _{k\neq i}p_{i} p_{k} = \sum _{i=1}^{L}\left(p_{i}\sum _{k\neq i}p_{k}\right)=\sum _{i=1}^{L}p_{i}(1-p_{i})=\sum _{i=1}^{L}p_{i}-\sum _{i=1}^{L}{p_{i}}^{2}=1-\sum _{i=1}^{L}{p_{i}}^{2}}$$
  
  where $p_i$ be the fraction of examples with the label $y_i$. We can then use this equation to calculate the **weighted gini index** for a feature $F$ as follows:
  
  $${\displaystyle \operatorname {Weighted\ Gini\ Index(F,D)} =\sum_{f \in values(F)}\frac{|D_f|}{|D|}G(D_f)}$$
  
  where $D_f$ is the set of all examples in $D$ where feature $F$ has the value $f$. 
  
  **IMPORTANT:** The `gini()` function below implements the $G(D)$ equation above; use it in the calculations of the **weighted gini index**. Notice that we are looking for the feature with the minimum **weighted gini index**.

In [13]:
df

,Outlook,Temperature,Humidity,Play Golf
0,Rainy,Hot,High,No
1,Overcast,Hot,High,Yes
2,Sunny,Mild,High,Yes
3,Sunny,Cool,Normal,No
4,Overcast,Cool,Normal,Yes
5,Rainy,Mild,High,No
6,Rainy,Cool,Normal,Yes
7,Sunny,Mild,Normal,Yes
8,Rainy,Mild,Normal,Yes
9,Overcast,Mild,High,Yes


In [47]:
g_outlook = 1 - ((4/11)**2 + (4/11)**2 + (2/11)**2)
g_outlook

0.7024793388429752

In [16]:
d_rainy = df[df['Outlook'] == 'Rainy']
d_overcast = df[df['Outlook'] == 'Overcast']
d_sunny = df[df['Outlook'] == 'Sunny']

gini_outlook = (
    (len(d_rainy) / len(df)) * gini(d_rainy) +
    (len(d_overcast) / len(df)) * gini(d_overcast) + 
    (len(d_sunny) / len(df)) * gini(d_sunny)
)
gini_outlook

0.30303030303030304

In [17]:
d_hot = df[df['Temperature'] == 'Hot']
d_mild = df[df['Temperature'] == 'Mild']
d_cool = df[df['Temperature'] == 'Cool']

gini_temperature = (
    (len(d_hot) / len(df)) * gini(d_hot) +
    (len(d_mild) / len(df)) * gini(d_mild) + 
    (len(d_cool) / len(df)) * gini(d_cool)
)
gini_temperature

0.38787878787878777

In [24]:
d_high = df[df['Humidity'] == 'High']
d_normal = df[df['Humidity'] == 'Normal']

gini_humidity = (
    (len(d_high) / len(df)) * gini(d_high) +
    (len(d_normal) / len(df)) * gini(d_normal)
)
gini_humidity

0.36969696969696964

In [41]:
p = df[['Outlook']].iloc[:,-1].value_counts() / len(df)
gini_imp = 1 - (p**2).sum()
df[['Outlook']].iloc[:,-1].value_counts() / len(df)

Rainy       0.363636
Overcast    0.363636
Sunny       0.272727
Name: Outlook, dtype: float64

In [42]:
gini_imp

0.6611570247933884

In [33]:
gini(df[['Outlook']])

0.6611570247933884

In [3]:
def gini(df):
    p = df.iloc[:, -1].value_counts() / len(df)
    return 1 - (p**2).sum()

In [30]:
def best_feature(df):
    features = df.columns[:-1]
    print('features', features)
    info = pd.DataFrame({'feature': features})
    print('info', info)
    
    
    info['weighted_gini'] = [gini(df[[feature]]) for feature in features]
    print(info)
    return info['feature'][info['weighted_gini'].argmin()]

In [31]:
best_feature(df)

features Index(['Outlook', 'Temperature', 'Humidity'], dtype='object')
info        feature
0      Outlook
1  Temperature
2     Humidity
       feature  weighted_gini
0      Outlook       0.661157
1  Temperature       0.644628
2     Humidity       0.495868


'Humidity'

In [5]:
tree = pd.Series(dtype=object)

def make_tree(df, node, feature=None):
    if feature is None:
        feature = best_feature(df)
        
    node[feature] = pd.Series(dtype=object)
    
    fvalues = df[feature].unique()
    for v in fvalues:
        d = df[df[feature] == v]
        n_classes = len(d.iloc[:, -1].unique())
        if n_classes == 1:
            node[feature][v] = ('L', d.iloc[:, -1].iloc[0])
        elif n_classes > 1:
            d = d.drop([feature], axis=1)
            if len(d.columns) == 1:
                node[feature][v] = ('L', d.iloc[:, -1].mode()[0])
            else:
                next_best_feature = best_feature(d)
                node[feature][v] = pd.Series(dtype=object)
                make_tree(d, node[feature][v], next_best_feature)

In [6]:
def print_tree(name, tree, d=1):
    for f in tree.index:
        if isinstance(tree[f], tuple):
            print('   ' * d, f, ' => ', tree[f], sep='')
        else:
            print('   ' * d, f, ': ', sep='')
            print_tree(f, tree[f], d + 1)

In [22]:
make_tree(df, tree)

* **(10 points)** Display the resulting tree. You can manually draw it out.

In [23]:
print_tree('', tree)

   Humidity: 
      High: 
         Temperature: 
            Hot: 
               Outlook: 
                  Rainy => ('L', 'No')
                  Overcast => ('L', 'Yes')
            Mild: 
               Outlook: 
                  Sunny => ('L', 'Yes')
                  Rainy => ('L', 'No')
                  Overcast => ('L', 'Yes')
      Normal: 
         Temperature: 
            Cool: 
               Outlook: 
                  Sunny => ('L', 'No')
                  Overcast => ('L', 'Yes')
                  Rainy => ('L', 'Yes')
            Mild => ('L', 'Yes')
            Hot => ('L', 'Yes')
   Outlook: 
      Rainy: 
         Temperature: 
            Hot => ('L', 'No')
            Mild: 
               Humidity: 
                  High => ('L', 'No')
                  Normal => ('L', 'Yes')
            Cool => ('L', 'Yes')
      Overcast => ('L', 'Yes')
      Sunny: 
         Temperature: 
            Mild => ('L', 'Yes')
            Cool => ('L', 'No')


* **(15 points)** Test your tree using the following examples:

In [9]:
def predict(unseen, node=None):
    if unseen.ndim == 1:
        if node is None:
            node = tree
            
        feature = node.index[0]
        children = node[feature]
        value = unseen[feature]

        for c in children.index:
            if c == value:
                if isinstance(children[c], tuple):
                    return children[c][1]
                else:
                    return predict(unseen, children[c])
    else:
        return np.array([predict(unseen.iloc[i,:]) for i in range(len(unseen))])

In [10]:
# Test scenario 1: [Overcast, Cool, Normal]
test1 = pd.Series(['Overcast', 'Cool', 'Normal'], index=['Outlook', 'Temperature', 'Humidity'])
print(test1)
predict(test1)

Outlook        Overcast
Temperature        Cool
Humidity         Normal
dtype: object


'Yes'

In [11]:
# Test scenario 2: [Rainy, Mild, High]
test2 = pd.Series(['Rainy', 'Mild', 'High'], index=['Outlook', 'Temperature', 'Humidity'])
print(test2)
predict(test2)

Outlook        Rainy
Temperature     Mild
Humidity        High
dtype: object


'No'

In [12]:
# Test scenario 3: [Sunny, Hot, Normal]
test3 = pd.Series(['Sunny', 'Hot', 'Normal'], index=['Outlook', 'Temperature', 'Humidity'])
print(test3)
predict(test3)

Outlook         Sunny
Temperature       Hot
Humidity       Normal
dtype: object


'Yes'